# Part 04



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gc

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

!pip install koreanize-matplotlib
import koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.7 MB/s eta 0:00:00


In [3]:
pip install factor_analyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor_analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42655 sha256=4c1aeef3f7ace6cdaf75552c72f102b603f13b6b01d6498c58a26f069e520542
  Stored in directory: /root/.cache/pip/wheels/a2/af/06/f4d4ed4d9d714fda437fb1583629417319603c2266e7b233cc
Successfully built factor_analyzer


In [4]:
df = pd.read_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/dataset/whisky_eda.csv')
df.columns = df.columns.str.lower()
print(df.shape)
df.head()

(1083, 51)


,name,whisky_type,volume(cl),alcohol(%),alcohol_density,price(£),price_per_cl,style_body,style_richness,style_smoke,style_sweetness,country,region,casktype,cask_group,bottler,bottler_group,is_independent,colouring,age,vintage,bottling_decade,is_vintage,has_age,has_bottling_decade,is_nas,characters,nose,palate,finish,food_paring,reviews,is_no_note,url,strong_smoke,style_missing,flavour_note,smoky_family,fruity_family,floral_family,herbal_family,sweet_family,nutty_family,spicy_family,woody_family,earthy_family,malty_family,creamy_family,wine_family,marine_family,other_family
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,blended,70.0,46.0,3220.0,34.25,0.489286,3.0,3.0,0.0,3.0,scotland,unknown_scotland,not specified,unknown,NaN,thompson bros,1,False,6.0,NaN,NaN,0,1,0,0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee","Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...",NaN,"Stunning stuff for the money, might well be my...",0,https://www.thewhiskyexchange.com/p/67983/blen...,0,0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffe...",0,1,0,0,1,0,1,0,0,0,0,0,0,0
1,Johnnie Walker Blue Label,blended,70.0,40.0,2800.0,158.00,2.257143,3.0,3.0,1.0,2.0,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,True,NaN,NaN,NaN,0,0,0,1,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,"Octopus, Monkfish, Fondue",Un elixir. || Excellent Whisky. Smooth on the ...,0,https://www.thewhiskyexchange.com/p/552/johnni...,0,0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...",0,0,1,0,1,1,1,1,1,1,0,0,0,1
2,The Big Ben,blended,50.0,40.0,2000.0,23.50,0.470000,NaN,NaN,NaN,NaN,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,True,NaN,NaN,NaN,0,0,0,1,no description,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,NaN,Very pleased with the whiskey. Highly recommend,0,https://www.thewhiskyexchange.com/p/18213/the-...,0,1,"Medium weight, malty nose, with hints of orang...",0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,J&B Rare,blended,70.0,40.0,2800.0,25.25,0.360714,NaN,NaN,NaN,NaN,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,True,NaN,NaN,NaN,0,0,0,1,no description,no description,no description,no description,NaN,Very nice || Was delighted to find this whisky...,1,https://www.thewhiskyexchange.com/p/544/jb-rare,0,1,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Dimple 15 Year OldLitre,blended,100.0,43.0,4300.0,69.95,0.699500,3.0,3.0,0.0,3.0,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,Unknown,15.0,NaN,NaN,0,1,0,0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",no description,no description,no description,NaN,NaN,0,https://www.thewhiskyexchange.com/p/79846/dimp...,0,0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",0,0,0,0,1,0,1,0,0,0,0,0,0,0


# Flavour 분류

## 𝟭) 스타일 지표

📎  <결과>

  ＃ 스타일 지표 분포 특성 <br>

   위스키의 기본 스타일 지표는 대부분 중간 강도와 균형 중심에 분포한다.
  대부분의 제품은 medium·round·non_smoke·balanced 조합을 이루며,
  strong smoke·dry 같은 개성적 스타일은 극히 적다.

      •	주요 패턴: medium body, round richness, non_smoke, balanced sweetness
	  •	희귀 패턴: strong smoke, dry sweetness


  ＃ Flavour Pattern(스타일 세트)별 가격 특성 <br>

  flavour_pattern별 가격 분석 결과, 특정 스타일 조합만 고가 또는 저가로 몰리는 현상은 없다.
  각 패턴은 넓은 가격대를 포함하며, 맛 패턴은 가격대를 설명하는 요인이 아니다.

      •	동일 패턴 안에서도 저가~고가가 모두 존재
	  •	style 조합은 가격 변동을 유의미하게 설명하지 못함
  



---




< 세그먼트 전 지표 정의 >

- 각 스타일은 0-5 척도로 구성되어 있고 각 숫자는 다음을 의미한다고 가정함. <br>

      	0: 거의 없음 / 특징이 아주 희미함
      	1: 아주 약하게 느껴짐
      	2: 은근히 느껴지지만, 메인은 아님
      	3: 확실히 느껴지는 수준(중간)
      	4: 꽤 강하게 존재감 있음
      	5: 이 축이 완전 메인 캐릭터

- Body, Richness, Sweetness

    0-1 light,lean,dry / 2-3 medium,round,balanced / 4-5 full,rich,sweet
- Smoke

    0 non / 1-2 light smoke / 3-4 smoky / 5 high_peat

In [5]:
# style 지표 존재하는 행
df_style = df[df['style_missing'] == 0]

style_cols = ['style_body', 'style_richness', 'style_smoke', 'style_sweetness']

# Body / Richness / Sweetness
def map_3level(x, low_label, mid_label, high_label):
    if 0 <= x <= 1:
        return low_label
    elif 2 <= x <= 3:
        return mid_label
    elif 4 <= x <= 5:
        return high_label
    else:
        return np.nan

def map_body(x):
    return map_3level(x, "light", "medium", "full")

def map_richness(x):
    return map_3level(x, "lean", "round", "rich")

def map_sweetness(x):
    return map_3level(x, "dry", "balanced", "sweet")

# --------------------------------------------------
# Smoke
def map_smoke(x):
    if x == 0:
        return "non"
    elif 1 <= x <= 2:
        return "light_smoke"
    elif 3 <= x <= 4:
        return "smoky"
    elif x == 5:
        return "high_peat"
    else:
        return np.nan

# 각 축별 태그 컬럼 생성
df_style["body_tag"] = df_style["style_body"].apply(map_body)
df_style["richness_tag"] = df_style["style_richness"].apply(map_richness)
df_style["smoke_tag"] = df_style["style_smoke"].apply(map_smoke)
df_style["sweetness_tag"] = df_style["style_sweetness"].apply(map_sweetness)

# 네 가지를 합쳐서 하나의 패턴 문자열로 만들기
df_style["flavour_pattern"] = (
    df_style["body_tag"] + "_"
    + df_style["richness_tag"] + "_"
    + df_style["smoke_tag"] + "_"
    + df_style["sweetness_tag"]
)

In [6]:
tag_cols = [col for col in df_style.columns if col.endswith("_tag")]

all_list = []

for col in tag_cols:
    print(f"<{col}>")
    temp = df_style[col].value_counts().reset_index()
    temp.columns = ["tag", "count"]
    temp["feature"] = col   # 어떤 feature인지 추가
    display(temp)
    print("\n------------------------------\n")

    all_list.append(temp)

df_all = pd.concat(all_list, ignore_index=True)
fig = px.bar(
    df_all,
    x="tag",
    y="count",
    color="feature",
    barmode="group",
    title="Style Tag Distribution Comparison"
)

fig.show()

<body_tag>


,tag,count,feature
0,medium,424,body_tag
1,full,185,body_tag
2,light,9,body_tag



------------------------------

<richness_tag>


,tag,count,feature
0,round,384,richness_tag
1,rich,192,richness_tag
2,lean,42,richness_tag



------------------------------

<smoke_tag>


,tag,count,feature
0,non,443,smoke_tag
1,light_smoke,112,smoke_tag
2,smoky,63,smoke_tag



------------------------------

<sweetness_tag>


,tag,count,feature
0,balanced,500,sweetness_tag
1,sweet,106,sweetness_tag
2,dry,12,sweetness_tag



------------------------------



In [7]:
# 패턴별 수, 가격 분포 확인
print(f"{df_style['flavour_pattern'].value_counts().shape[0]} 가지 패턴 발견")
flavour_pattern_with_price = df_style.groupby('flavour_pattern').agg(
    count=('price(£)', 'size'),
    price_min=('price(£)', 'min'),
    price_mean=('price(£)', 'mean'),
    price_med = ('price(£)', 'median'),
    price_max=('price(£)', 'max')
).reset_index().sort_values('count', ascending=False)
flavour_pattern_with_price[:10]

35 가지 패턴 발견


,flavour_pattern,count,price_min,price_mean,price_med,price_max
30,medium_round_non_balanced,161,25.25,540.878882,67.750,24000.0
2,full_rich_non_balanced,64,39.95,315.187344,130.500,4500.0
27,medium_round_light_smoke_balanced,59,22.50,276.104068,60.250,9995.0
24,medium_rich_non_balanced,49,38.95,210.760204,74.950,3750.0
32,medium_round_non_sweet,42,17.95,80.684524,58.100,595.0
7,full_round_non_balanced,36,37.25,126.294444,77.600,710.0
18,medium_lean_non_balanced,30,25.50,85.805000,56.500,262.0
33,medium_round_smoky_balanced,26,22.75,414.328846,51.225,5000.0
3,full_rich_non_sweet,19,43.75,233.628947,215.000,945.0
8,full_round_non_sweet,17,37.50,183.982353,68.500,975.0


In [8]:
# 이상치 제거 후 가격 분포 확인
df_clean = df_style[(df_style['price(£)'] >= 20) & (df_style['price(£)'] <= 5000)]

flavour_pattern_with_price_cleaned = df_clean.groupby('flavour_pattern').agg(
    count=('price(£)', 'size'),
    price_min=('price(£)', 'min'),
    price_mean=('price(£)', 'mean'),
    price_med = ('price(£)', 'median'),
    price_max=('price(£)', 'max')
).reset_index().sort_values('count', ascending=False)

fig = px.box(
    df_clean,
    x='flavour_pattern',
    y='price(£)',
    points=False,
    title='Price Distribution by Flavour Pattern (Outliers Removed)'
)

fig.update_layout(
    xaxis={'categoryorder':'array',
           'categoryarray': flavour_pattern_with_price['flavour_pattern']},
    height=600
)
fig.update_xaxes(tickangle=90)

fig.show()

## 𝟐) Text 변수

- 앞서 Part 03, EDA 단계에서 character, palate, nose, finish 4가지에서 대표 flavour로 라벨링한 _family 변수를 이용해 확인했습니다.

<br>

📎  <결과>


< 전체 위스키 중 두 family가 동시에 등장한 비율 > <br> <br>
공출현(Co-occurrence) 비율 20% 이상인 조합을 기준으로 보면, <br>
현재 위스키 데이터셋은 세 가지 핵심 향미 축을 중심으로 구성돼 있음을 확인할 수 있다.

1. **Fruity - Sweet (57.3%)**
2. **Sweet - Spicy (41.4%) & Fruity - Spicy (39.3%)**
3. **Sweet - Woody (24.8%) & Fruity - Woody (21.3%)**

In [9]:
# 전체 제품 수
n_whisky = len(df_style)
family_cols = [col for col in df_style.columns if col.endswith('_family')]


# 1. 개별 flavour_family 등장 비율

family_stats = (
    df_style[family_cols]
    .sum()
    .rename('count')
    .to_frame()
)

family_stats['ratio'] = family_stats['count'] / n_whisky
family_stats = (
    family_stats
    .sort_values('ratio', ascending=False)
    .reset_index()
    .rename(columns={'index': 'family'})
)

print("=== Family별 등장 비율 ===")
display(family_stats)

=== Family별 등장 비율 ===


,family,count,ratio
0,sweet_family,485,0.784790
1,fruity_family,452,0.731392
2,spicy_family,324,0.524272
3,woody_family,177,0.286408
4,nutty_family,145,0.234628
5,herbal_family,81,0.131068
6,other_family,79,0.127832
7,floral_family,74,0.119741
8,earthy_family,55,0.088997
9,malty_family,50,0.080906


In [10]:
pd.set_option('display.max_colwidth', None)


# 자주 보이는 조합들
df_style['family_combo'] = (
    df_style[family_cols]
    .apply(lambda row: tuple(col.replace('_family', '')
                             for col in row.index[row == 1]),axis=1)
)

combo_counts = (
    df_style['family_combo']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'family_combo', 'family_combo': 'count'})
)


print("=== 자주 보이는 Family 조합들 ===")
display(combo_counts[:10])
print("\n\n=== 소수 Family 조합들 ===")
display(combo_counts[-10:])


=== 자주 보이는 Family 조합들 ===


,count,count
0,"(fruity, sweet, spicy)",58
1,"(fruity, sweet)",45
2,"(sweet, spicy)",25
3,"(fruity, sweet, nutty, spicy)",20
4,"(sweet,)",20
5,"(fruity, sweet, nutty)",20
6,"(fruity, sweet, woody)",19
7,"(fruity, spicy)",19
8,"(fruity, sweet, spicy, woody)",17
9,"(fruity,)",15




=== 소수 Family 조합들 ===


,count,count
187,"(fruity, floral, herbal)",1
188,"(fruity, herbal, sweet, spicy, woody, wine)",1
189,"(fruity, herbal, sweet, nutty, spicy, woody, malty, wine)",1
190,"(spicy, woody, earthy)",1
191,"(sweet, spicy, malty)",1
192,"(fruity, floral, sweet, woody, earthy, malty)",1
193,"(fruity, herbal, sweet, malty)",1
194,"(fruity, herbal, sweet, spicy, woody, creamy, other)",1
195,"(fruity, sweet, nutty, wine)",1
196,"(fruity, herbal, sweet, nutty, spicy, woody, other)",1


In [11]:
# one-hot 행렬
X_fam = df_style[family_cols].astype(int)

# 공출현 행렬: Fᵀ·F
co_mat = X_fam.T.dot(X_fam)   # shape: (n_family, n_family)

# 비율로도 보고 싶으면
co_mat_ratio = co_mat / n_whisky

print("=== Family Co-occurrence (count) ===")
display(co_mat)

print("=== Family Co-occurrence (ratio w.r.t all whiskies) ===")
display(co_mat_ratio)

# 예: fruity와 sweet가 같이 나오는 비율 확인
if 'fruity_family' in family_cols and 'sweet_family' in family_cols:
    fruity_sweet_ratio = co_mat_ratio.loc['fruity_family', 'sweet_family']
    print(f"\nfruity + sweet 동시 등장 비율: {fruity_sweet_ratio:.3f}")

=== Family Co-occurrence (count) ===


,smoky_family,fruity_family,floral_family,herbal_family,sweet_family,nutty_family,spicy_family,woody_family,earthy_family,malty_family,creamy_family,wine_family,marine_family,other_family
smoky_family,20,15,4,5,12,4,11,8,4,7,2,2,4,10
fruity_family,15,452,57,70,354,99,243,132,41,43,39,25,3,69
floral_family,4,57,74,14,63,18,31,24,9,15,9,2,2,18
herbal_family,5,70,14,81,66,12,46,28,11,17,17,7,2,25
sweet_family,12,354,63,66,485,117,256,153,42,39,39,24,3,72
nutty_family,4,99,18,12,117,145,65,33,12,10,13,12,1,19
spicy_family,11,243,31,46,256,65,324,110,28,30,21,25,1,57
woody_family,8,132,24,28,153,33,110,177,25,23,21,17,1,49
earthy_family,4,41,9,11,42,12,28,25,55,10,6,3,2,16
malty_family,7,43,15,17,39,10,30,23,10,50,9,4,2,23


=== Family Co-occurrence (ratio w.r.t all whiskies) ===


,smoky_family,fruity_family,floral_family,herbal_family,sweet_family,nutty_family,spicy_family,woody_family,earthy_family,malty_family,creamy_family,wine_family,marine_family,other_family
smoky_family,0.032362,0.024272,0.006472,0.008091,0.019417,0.006472,0.017799,0.012945,0.006472,0.011327,0.003236,0.003236,0.006472,0.016181
fruity_family,0.024272,0.731392,0.092233,0.113269,0.572816,0.160194,0.393204,0.213592,0.066343,0.069579,0.063107,0.040453,0.004854,0.111650
floral_family,0.006472,0.092233,0.119741,0.022654,0.101942,0.029126,0.050162,0.038835,0.014563,0.024272,0.014563,0.003236,0.003236,0.029126
herbal_family,0.008091,0.113269,0.022654,0.131068,0.106796,0.019417,0.074434,0.045307,0.017799,0.027508,0.027508,0.011327,0.003236,0.040453
sweet_family,0.019417,0.572816,0.101942,0.106796,0.784790,0.189320,0.414239,0.247573,0.067961,0.063107,0.063107,0.038835,0.004854,0.116505
nutty_family,0.006472,0.160194,0.029126,0.019417,0.189320,0.234628,0.105178,0.053398,0.019417,0.016181,0.021036,0.019417,0.001618,0.030744
spicy_family,0.017799,0.393204,0.050162,0.074434,0.414239,0.105178,0.524272,0.177994,0.045307,0.048544,0.033981,0.040453,0.001618,0.092233
woody_family,0.012945,0.213592,0.038835,0.045307,0.247573,0.053398,0.177994,0.286408,0.040453,0.037217,0.033981,0.027508,0.001618,0.079288
earthy_family,0.006472,0.066343,0.014563,0.017799,0.067961,0.019417,0.045307,0.040453,0.088997,0.016181,0.009709,0.004854,0.003236,0.025890
malty_family,0.011327,0.069579,0.024272,0.027508,0.063107,0.016181,0.048544,0.037217,0.016181,0.080906,0.014563,0.006472,0.003236,0.037217



fruity + sweet 동시 등장 비율: 0.573


In [12]:
thr = 0.20  #기준

pairs = []
idx = co_mat_ratio.index
cols = co_mat_ratio.columns

for i in range(len(idx)):
    for j in range(i+1, len(cols)):
        val = co_mat_ratio.iloc[i, j]
        if val >= thr:
            pairs.append({
                'family_1': idx[i],
                'family_2': cols[j],
                'ratio': val
            })

strong_pairs = (
    pd.DataFrame(pairs)
    .sort_values('ratio', ascending=False)
    .reset_index(drop=True)
)

print(f"\n=== Co-occurrence pairs (ratio ≥ {thr:.2f}) ===")
display(strong_pairs)


=== Co-occurrence pairs (ratio ≥ 0.20) ===


,family_1,family_2,ratio
0,fruity_family,sweet_family,0.572816
1,sweet_family,spicy_family,0.414239
2,fruity_family,spicy_family,0.393204
3,sweet_family,woody_family,0.247573
4,fruity_family,woody_family,0.213592



# 진열 프레임



| **Richness ↑ / Flavour →** | **Non-Smoky (Fruity–Spicy–Woody...)** | **Smoky** |
|----------------------------|-------------------------------------|-----------|
| **Full / Rich**            | Lean → Full                         | Lean → Full (진열 규칙)|
| **Round**                  | Lean → Full                         | Lean → Full |
| **Light**                  | Lean → Full                         | Lean → Full |


<br><br>
☑️ 진열 전략은 Richness(행) × Flavour Family(열)의 3×3 매트릭스 구조로 설계된다.<br>
열은 Non-smoky, Smoky의 네 가지 Flavour Direction으로 구성되며,<br>
위스키는 sweetness intensity(단맛의 절대 강도)보다 맛의 방향성(flavour direction) 에 따라 <br>분류·탐색하는 것이 더 적절하다는 점을 고려하여,<br>
데이터에서 주요하게 나타나는 향미 특성들을 기준으로 구분하였다.<br>
각 셀 내부에서는 Body 지표(lean→full)를 좌→우의 연속적인 스펙트럼으로 시각화하여 제품을 배치한다.<br>

이를 통해 고객은 다음 두 가지 축을 직관적으로 탐색할 수 있다.

	1.	바디감 (light–round–rich의 무게감 차이)
	2.	풍미 방향성 (Fruity / Spicy / Woody / ... / Smoky)

이 구조는 고객이 자신의 취향에 맞는 위스키를 빠르게 찾을 수 있도록 돕는 동시에,
제품 간 풍미 차이를 시각적으로 명확하게 전달하는 데 효과적이다.




In [13]:
pd.set_option('display.max_colwidth', 50)
df.head()

,name,whisky_type,volume(cl),alcohol(%),alcohol_density,price(£),price_per_cl,style_body,style_richness,style_smoke,style_sweetness,country,region,casktype,cask_group,bottler,bottler_group,is_independent,colouring,age,vintage,bottling_decade,is_vintage,has_age,has_bottling_decade,is_nas,characters,nose,palate,finish,food_paring,reviews,is_no_note,url,strong_smoke,style_missing,flavour_note,smoky_family,fruity_family,floral_family,herbal_family,sweet_family,nutty_family,spicy_family,woody_family,earthy_family,malty_family,creamy_family,wine_family,marine_family,other_family
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,blended,70.0,46.0,3220.0,34.25,0.489286,3.0,3.0,0.0,3.0,scotland,unknown_scotland,not specified,unknown,NaN,thompson bros,1,False,6.0,NaN,NaN,0,1,0,0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee","Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...",NaN,"Stunning stuff for the money, might well be my...",0,https://www.thewhiskyexchange.com/p/67983/blen...,0,0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffe...",0,1,0,0,1,0,1,0,0,0,0,0,0,0
1,Johnnie Walker Blue Label,blended,70.0,40.0,2800.0,158.00,2.257143,3.0,3.0,1.0,2.0,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,True,NaN,NaN,NaN,0,0,0,1,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,"Octopus, Monkfish, Fondue",Un elixir. || Excellent Whisky. Smooth on the ...,0,https://www.thewhiskyexchange.com/p/552/johnni...,0,0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...",0,0,1,0,1,1,1,1,1,1,0,0,0,1
2,The Big Ben,blended,50.0,40.0,2000.0,23.50,0.470000,NaN,NaN,NaN,NaN,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,True,NaN,NaN,NaN,0,0,0,1,no description,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,NaN,Very pleased with the whiskey. Highly recommend,0,https://www.thewhiskyexchange.com/p/18213/the-...,0,1,"Medium weight, malty nose, with hints of orang...",0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,J&B Rare,blended,70.0,40.0,2800.0,25.25,0.360714,NaN,NaN,NaN,NaN,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,True,NaN,NaN,NaN,0,0,0,1,no description,no description,no description,no description,NaN,Very nice || Was delighted to find this whisky...,1,https://www.thewhiskyexchange.com/p/544/jb-rare,0,1,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Dimple 15 Year OldLitre,blended,100.0,43.0,4300.0,69.95,0.699500,3.0,3.0,0.0,3.0,scotland,unknown_scotland,not specified,unknown,NaN,distillery bottling,0,Unknown,15.0,NaN,NaN,0,1,0,0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",no description,no description,no description,NaN,NaN,0,https://www.thewhiskyexchange.com/p/79846/dimp...,0,0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",0,0,0,0,1,0,1,0,0,0,0,0,0,0


In [14]:
cols_to_add = [
    'body_tag', 'richness_tag', 'smoke_tag', 'sweetness_tag',
    'flavour_pattern', 'family_combo'
]

df[cols_to_add] = df_style[cols_to_add]

# 데이터 저장하기

In [15]:
df.to_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/dataset/whisky_framed.csv',index=False)